In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from PIL import Image
import requests
from sklearn.preprocessing import OneHotEncoder
import os
import cv2 as cv
import rasterio
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
f = open('bandwise_minmax.pkl', 'rb')
minmax = pickle.load(f)
f.close()

def makedir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def norm(band, index):
  band_min, band_max = minmax[index]['min'], minmax[index]['max']
  return ((band - band_min)/(band_max - band_min))

def get_image(image_path):
  src = rasterio.open(image_path, 'r')
  S_images=src.read()
  c0 = norm(S_images[0].astype(np.float32), 0)
  c1 = norm(S_images[1].astype(np.float32), 1)
  c2 = norm(S_images[2].astype(np.float32), 2)
  c3 = norm(S_images[3].astype(np.float32), 3)
  c4 = norm(S_images[4].astype(np.float32), 4)
  c5 = norm(S_images[5].astype(np.float32), 5)
  image = np.dstack((c0,c1,c2,c3,c4,c5))
  src.close()
  return image

def process_image(satellite_image):
  satellite_image = satellite_image[1:-1, 1:-1, :]
  h,w,c = satellite_image.shape
  if w < h:
    satellite_image = np.transpose(satellite_image, (1, 0, 2))
  resized_image = cv.resize(satellite_image, (25, 25), interpolation=cv.INTER_CUBIC)
  return resized_image

def get_age(year, location, time_point, planting_date):
  collection_date = doc[str(year)][location][time_point]
  planting_date = pd.to_datetime(planting_date)
  collection_date = pd.to_datetime(collection_date)
  return (collection_date-planting_date).days

In [3]:
test_2023_split = pd.read_csv('data_processed/test_2023.csv')

In [4]:
f = open('data_processed/doc_2023.pkl', 'rb')
doc_2023 = pickle.load(f)
f.close()
f = open('data_processed/val_2023.pkl', 'rb')
data_split = pickle.load(f)
f.close()

In [5]:
doc = {}
doc['2023'] = doc_2023

Steps in data preprocessing:


*   Remove Scottsbluff from training set (scottsbluff plot length is different from all other data)
*   Remove rows with nan
*   Remove genotype (2023 val set has two genotypes that are new) and irrigationProvided
*   Divide yield by max yield. Store max yield in metadata
*   Convert location, N_level into one hot encoded form
*   Get the satellite images for each row. Transpose image if width is shorter than height. Reside to 25 by 25. And min max normalize each channel of the image.
*   For each satellite image, Calculate age by finding difference between planting date and time point. Divide age by 365.
*   Make a list of data samples and another list of yields
*   Divide the list into training vs validation
*   Save as numpy array or pickle file



In [7]:
all_data = test_2023_split

In [8]:
all_data.head()

,year,location,irrigationProvided,experiment,range,row,N_level,genotype,planting_date,location_Ames,location_Crawfordsville,location_Lincoln,location_MOValley,N_level_High,N_level_Low,N_level_Medium
0,2023,Ames,0,2315,9,15,Low,LH185 X LH145,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2023,Ames,0,2315,7,11,Low,PHN46 X PHK56,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2023,Ames,0,2315,4,17,Low,PHN46 X W606S,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2023,Ames,0,2315,20,17,Low,B73 X PHZ51,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2023,Ames,0,2315,19,13,Low,LH145 X LH162,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
with open('processed_data_minmax_normalized_trimmed.pkl', 'rb') as f:
  processed_data = pickle.load(f)

In [10]:
max_yield = processed_data['metadata']['max_yield']

In [11]:
all_data = all_data.drop(columns=['genotype','irrigationProvided'])

In [12]:
all_data.head()

,year,location,experiment,range,row,N_level,planting_date,location_Ames,location_Crawfordsville,location_Lincoln,location_MOValley,N_level_High,N_level_Low,N_level_Medium
0,2023,Ames,2315,9,15,Low,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2023,Ames,2315,7,11,Low,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2023,Ames,2315,4,17,Low,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2023,Ames,2315,20,17,Low,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2023,Ames,2315,19,13,Low,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
images = []
numerical_features = []

for index, row in tqdm(all_data.iterrows()):

  location = row['location']
  year = row['year']
  if year == 2023:
    year = '2023_test'
  experiment = row['experiment']
  if experiment == 'Hyrbrids':
    experiment = 'Hybrids'
  range = row['range']
  row_ = row['row']
  planting_date = row['planting_date']
  time_points = sorted(os.listdir(os.path.join('satellite_images', str(year), location)))

  new_images = []
  new_numerical_features = []  
  for time_point in time_points:
    folder_path = os.path.join('satellite_images', str(year), location, time_point)
    image_name = f'{location}-{time_point}-{experiment}_{range}_{row_}.TIF'
    image_path = os.path.join(folder_path, image_name)
    if os.path.exists(image_path):
      satellite_image = get_image(image_path)
      processed_image = process_image(satellite_image)
      age = get_age(row['year'], location, time_point, planting_date)
      age = age/365
      new_images.append(processed_image)
      new_numerical_features.append([row['location_Ames'],
                     row['location_Crawfordsville'],
                     row['location_Lincoln'],
                     row['location_MOValley'],
                     row['N_level_High'],
                     row['N_level_Low'],
                     row['N_level_Medium'],
                     age])
      print('found ',image_path)  
    else:
      print('not_found', image_path)
  images.append(new_images)
  numerical_features.append(new_numerical_features)  

images = np.array(images)
numerical_features = np.array(numerical_features)

5it [00:00, 20.64it/s]

found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2315_9_15.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2315_9_15.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2315_9_15.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2315_7_11.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2315_7_11.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2315_7_11.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2315_4_17.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2315_4_17.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2315_4_17.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2315_20_17.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2315_20_17.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2315_20_17.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2315_19_13.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2315_19_13.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2315_19_13.TIF
foun

14it [00:00, 32.64it/s]

found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2315_5_11.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2315_5_11.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2315_6_13.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2315_6_13.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2315_6_13.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2315_14_17.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2315_14_17.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2315_14_17.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2315_22_15.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2315_22_15.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2315_22_15.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2315_7_15.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2315_7_15.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2315_7_15.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2315_9_11.TIF
foun

22it [00:00, 34.17it/s]

...

found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2335_25_55.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2335_25_55.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2335_41_59.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2335_41_59.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2335_41_59.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2335_41_55.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2335_41_55.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2335_41_55.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2335_28_61.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2335_28_61.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2335_28_61.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2335_44_57.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2335_44_57.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2335_44_57.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2335_31_59

283it [00:09, 29.69it/s]

found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2335_34_55.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2335_34_55.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2335_34_55.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2335_25_61.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2335_25_61.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2335_25_61.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2335_40_61.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2335_40_61.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2335_40_61.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2335_45_57.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2335_45_57.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2335_45_57.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2335_35_61.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2335_35_61.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2335_35_61

288it [00:09, 31.02it/s]

found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2335_37_55.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2335_35_55.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2335_35_55.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2335_35_55.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2335_31_57.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2335_31_57.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2335_31_57.TIF
found  satellite_images\2023_test\Ames\TP1\Ames-TP1-2335_40_59.TIF
found  satellite_images\2023_test\Ames\TP2\Ames-TP2-2335_40_59.TIF
found  satellite_images\2023_test\Ames\TP3\Ames-TP3-2335_40_59.TIF


In [14]:
data_dict = {'images': images,
             'numerical_features': numerical_features,
             'metadata': processed_data['metadata']}

In [15]:
f = open('processed_test_data_minamx_normalized_trimmed.pkl', 'wb')
pickle.dump(data_dict, f)
f.close()